## Pull In Fly Ball Data for a Given Date

In [166]:
fly_balls = {}

In [167]:
import datetime
import mlbapi
string_date = '2021-10-27'
base = datetime.date(int(string_date.split('-')[0]),
                     int(string_date.split('-')[1]),
                     int(string_date.split('-')[2]))
date_list = [base]
date_list

[datetime.date(2021, 10, 27)]

In [168]:
import time

count = 0 
fly_count = 0
for date_string in date_list:
    print(len(fly_balls))
    game_list = mlbapi.schedule(date=date_string).dates[0].games
    game_list = [x for x in game_list if ((x.status.detailed_state != 'Suspended') and (x.status.detailed_state != 'Postponed'))]
    if len(game_list) > 0:
        time.sleep(3)
        for game in game_list:
            plays = mlbapi.play_by_play(game.game_pk)
            for play in plays['allPlays']:
                if play['result']['event'] in ['Sac Fly', 'Forceout', 'Fielders Choice', 'Fielders Choice Out', 'Double Play', 'Single', 'Double', 'Triple', 'Field Error', 'Pop Out', 'Flyout', 'Lineout']:
                    if play['playEvents'][-1]['hitData']['trajectory'] != 'ground_ball':
                        try:
                            if play['playEvents'][-1]['hitData']['location'] in ['7','8','9']:
                                play_show = play['playEvents'][-1]['hitData']
                                play_show['results'] = play['result']['event']
                                play_show['x'] = play_show['coordinates']['coordX']
                                play_show['y'] = play_show['coordinates']['coordY']
                                play_show['hitter'] = play['matchup']['batter']['fullName']
                                i = play['result']['description']
                                if ': ' in i:
                                    i = i.replace(i.split(': ')[0]+': ', '')
                                if '.' in play_show['hitter']:
                                    i = i.replace(play_show['hitter'].replace('.', '.    '), play_show['hitter'])
                                    i = i.replace(play_show['hitter'].replace('.', '.   '), play_show['hitter'])
                                    i = i.replace(play_show['hitter'].replace('.', '.  '), play_show['hitter'])
                                    i = i.replace(play_show['hitter'].replace('.', '. '), play_show['hitter'])
                                if '  ' in i:
                                    i = i.split('  ')[0]    
                                if 'deflected' in i:
                                    i = i.split(', deflected')[0]
                                i = i.replace('Jr.flies', 'Jr. flies')
                                i = i.replace('Jr.pops', 'Jr. pops')
                                i = i.replace('Jr.lines', 'Jr. lines')
                                i = i.replace('Jr.singles', 'Jr. singles')
                                i = i.replace('Jr.doubles', 'Jr. doubles')
                                i = i.replace('Jr.triples', 'Jr. triples')
                                i = i.replace('Jr.reaches', 'Jr. reaches')
                                i = i.replace('Sr.flies', 'Sr. flies')
                                i = i.replace('Sr.pops', 'Sr. pops')
                                i = i.replace('Sr.lines', 'Sr. lines')
                                i = i.replace('Sr.singles', 'Sr. singles')
                                i = i.replace('Sr.doubles', 'Sr. doubles')
                                i = i.replace('Sr.triples', 'Sr. triples')
                                i = i.replace('Sr.reaches', 'Sr. reaches')
                                play_show['description'] = i
                                play_show['inning'] = play['about']['inning']
                                play_show['halfInning'] = play['about']['halfInning']
                                if play_show['halfInning'] == 'top':
                                    play_show['away_score'] = play['result']['awayScore'] - play['result']['rbi']
                                    play_show['home_score'] = play['result']['homeScore']
                                else:
                                    play_show['away_score'] = play['result']['awayScore']
                                    play_show['home_score'] = play['result']['homeScore'] - play['result']['rbi']
                                play_show['playId'] = play['playEvents'][-1]['playId']
                                play_show['stadium'] = game.teams.home.team.name
                                play_show['away_team'] = game.teams.away.team.name
                                play_show['outs'] = play['count']['outs']
                                play_show['date'] = string_date
                                play_show['hitter'] = play['matchup']['batter']['link'].split('/')[-1]
                                for fi in range(len(play['runners'])):
                                    try:
                                        play_show['fielder'] = play['runners'][fi]['credits'][0]['player']['link'].split('/')[-1]
                                    except:
                                        continue
                                fly_count += 1
                                fly_balls[fly_count] = play_show
                        except:
                            continue

0
https://statsapi.mlb.com/api/v1/schedule {'date': datetime.date(2021, 10, 27), 'sportId': 1}


In [169]:
import pandas as pd

dff = pd.DataFrame.from_dict(fly_balls, orient='index')
plays = dff.copy()
plays.head()

,launchSpeed,launchAngle,totalDistance,trajectory,hardness,location,coordinates,results,x,y,...,inning,halfInning,away_score,home_score,playId,stadium,away_team,outs,date,fielder
1,103.7,8.0,224.0,line_drive,medium,9,"{'coordX': 175.34, 'coordY': 118.63}",Single,175.34,118.63,...,1,top,0,0,aea0e86e-9204-4709-ad20-1f1c7bb50141,Houston Astros,Atlanta Braves,2,2021-10-27,663656
2,83.8,17.0,205.0,line_drive,medium,7,"{'coordX': 49.47, 'coordY': 130.86}",Double,49.47,130.86,...,1,bottom,0,0,844575ad-f70e-401c-be73-8d690cec7cad,Houston Astros,Atlanta Braves,0,2021-10-27,592696
3,99.2,25.0,368.0,fly_ball,medium,8,"{'coordX': 82.87, 'coordY': 57.1}",Flyout,82.87,57.10,...,1,bottom,0,0,3cb913d5-89d9-43a8-81e9-6b926c59cbfa,Houston Astros,Atlanta Braves,1,2021-10-27,594807
4,102.3,20.0,373.0,line_drive,hard,8,"{'coordX': 122.69, 'coordY': 48.7}",Sac Fly,122.69,48.70,...,1,bottom,0,0,311204fb-f7c8-44f0-b0d5-df67b75860ea,Houston Astros,Atlanta Braves,2,2021-10-27,594807
5,92.0,18.0,300.0,line_drive,medium,8,"{'coordX': 116.7, 'coordY': 79.62}",Lineout,116.70,79.62,...,2,top,0,1,9ff9d605-8290-493e-99bb-112a1f11c50a,Houston Astros,Atlanta Braves,2,2021-10-27,642350


## Calculate Catch Probability from Saved Model

In [170]:
import pickle
import math
with open('model5.pkl', 'rb') as f:
    logisticRegr = pickle.load(f)
dff = dff[dff['launchSpeed'].notna()]
dff = dff[dff['launchAngle'].notna()].reset_index(drop=True)
dff['launchAngle'] = dff['launchAngle'].astype(float)
dff['launchSpeed'] = dff['launchSpeed'].astype(float)
dff['launchSpeed_og'] = dff['launchSpeed'].copy()
dff['launchSpeed'] = dff['launchSpeed'] * .44704
time_list = []
for i in range(len(dff)):
    time_list.append((dff['launchSpeed'][i]*math.sin(dff['launchAngle'][i]*(math.pi/180))+math.sqrt((dff['launchSpeed'][i]*math.sin(dff['launchAngle'][i]*(math.pi/180)))**2 + (2 * 9.8 * 1))) / 9.8)
dff2 = pd.read_csv('22_input.csv')
centroids = dff2.groupby(['stadium', 'location'])[['x', 'y']].mean().reset_index()
centroid_map_x = dict(zip(centroids['stadium']+centroids['location'].astype(str), centroids['x']))
centroid_map_y = dict(zip(centroids['stadium']+centroids['location'].astype(str), centroids['y']))
dff['hang_time'] = time_list
dff['out'] = 1
dff.loc[dff['results'].isin(['Single', 'Double', 'Triple', 'Field Error']), 'out'] = 0
dff['centroid_x'] = (dff['stadium']+dff['location'].astype(str)).map(centroid_map_x)
dff['centroid_y'] = (dff['stadium']+dff['location'].astype(str)).map(centroid_map_y)
distance_list = []
for i in range(len(dff)):
    distance_list.append( math.sqrt(  (dff['centroid_x'][i]-dff['x'][i])**2  + (dff['centroid_y'][i]-dff['y'][i])**2))
dff['distance'] = distance_list
dff = dff[dff['hang_time'].notna()]
dff = dff[dff['distance'].notna()]
dff['catch_odds'] = logisticRegr.predict_proba(dff[['distance', 'hang_time']])[::, 1]
plays = dff.copy()

C:\Users\btmyr\anaconda3\Lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator KNeighborsClassifier from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [171]:
all_plays = [[x, plays.loc[x]['description'], plays.loc[x]['catch_odds']] for x in range(len(plays))]
all_plays = pd.DataFrame(all_plays, columns = ['key', 'description', 'catch_odds'])
all_plays

,key,description,catch_odds
0,0,Austin Riley singles on a line drive to right ...,0.02
1,1,Jose Altuve doubles (1) on a line drive to lef...,0.00
2,2,Michael Brantley flies out to center fielder A...,0.40
3,3,Alex Bregman out on a sacrifice fly to center ...,0.74
4,4,Adam Duvall lines out to center fielder Jose S...,0.55
5,5,Jose Altuve lines out to center fielder Adam D...,0.94
6,6,Austin Riley flies out to center fielder Jose ...,0.97
7,7,Adam Duvall flies out to right fielder Kyle Tu...,0.98
8,8,Yuli Gurriel flies out to left fielder Eddie R...,1.00
9,9,Freddie Freeman singles on a sharp line drive ...,0.05


In [174]:
key = 4
all_plays[all_plays['key'] == key]

,key,description,catch_odds
4,4,Adam Duvall lines out to center fielder Jose S...,0.55


In [181]:
import numpy as np
from PIL import Image
from scipy import interpolate
import matplotlib.pyplot as plt
import requests

tm_abbrev = {}
tm_abbrev['Pittsburgh Pirates'] = 'PIT'
tm_abbrev['Washington Nationals'] = 'WSH'
tm_abbrev['Toronto Blue Jays'] = 'TOR'
tm_abbrev['Boston Red Sox'] = 'BOS'
tm_abbrev['Baltimore Orioles'] = 'BAL'
tm_abbrev['Philadelphia Phillies'] = 'PHI'
tm_abbrev['Tampa Bay Rays'] = 'TB'
tm_abbrev['Miami Marlins'] = 'MIA'
tm_abbrev['Cincinnati Reds'] = 'CIN'
tm_abbrev['Kansas City Royals'] = 'KC'
tm_abbrev['Chicago White Sox'] = 'CHW'
tm_abbrev['Texas Rangers'] = 'TEX'
tm_abbrev['Colorado Rockies'] = 'COL'
tm_abbrev['Arizona Diamondbacks'] = 'ARI'
tm_abbrev['Minnesota Twins'] = 'MIN'
tm_abbrev['Oakland Athletics'] = 'OAK'
tm_abbrev['Los Angeles Dodgers'] = 'LAD'
tm_abbrev['New York Yankees'] = 'NYY'
tm_abbrev['San Diego Padres'] = 'SD'
tm_abbrev['Atlanta Braves'] = 'ATL'
tm_abbrev['Houston Astros'] = 'HOU'
tm_abbrev['Chicago Cubs'] = 'CHC'
tm_abbrev['Milwaukee Brewers'] = 'MIL'
tm_abbrev['Seattle Mariners'] = 'SEA'
tm_abbrev['Los Angeles Angels'] = 'LAA'
tm_abbrev['Detroit Tigers'] = 'DET'
tm_abbrev['Cleveland Guardians'] = 'CLE'
tm_abbrev['San Francisco Giants'] = 'SF'
tm_abbrev['New York Mets'] = 'NYM'
tm_abbrev['St. Louis Cardinals'] = 'STL'

example = plays.loc[key]
hit_speed = 99 * 0.44704
hit_speed = example['launchSpeed']
playId = example['playId']
hit_angle = example['launchAngle']
description = example['description']+' ('+example['date']+')'
hit_location = example['location']
hitter = example['hitter']
fielder = example['fielder']
outs = str(example['outs'])
inning = example['halfInning'].replace('top', '⇧').replace('bottom', '⇩')+str(example['inning'])+'th'
stadium = example['stadium']
result_play = example['results']
x = example['x']
y = example['y']
away_team = tm_abbrev[example['away_team']]
home_team = tm_abbrev[example['stadium']]
away_score = example['away_score']
home_score = example['home_score']
if x == 125:
    x = 124.97
max_h = (((hit_speed**2)*(math.sin(hit_angle*(math.pi/180))**2))/19.6)+1
mid_x = (125+x)/2
mid_y = (222+y)/2 - max_h
end_x = 125
end_y = 220
x2 = np.linspace(x, end_x, 100)
if (x < mid_x) & (mid_x < end_x):
    y2 = interpolate.pchip_interpolate([x,mid_x,end_x], [y, mid_y, end_y], x2)
elif (x > mid_x) & (mid_x > end_x):
    y2 = interpolate.pchip_interpolate([end_x,mid_x,x], [end_y, mid_y, y], x2)
x2 = np.linspace(x, end_x, 100)
z2 = np.linspace(0, max_h*8, 50)
array2 = np.linspace(max_h*8, 0, 50)

# Appending both arrays using Append method
z2 = np.append(z2, array2)
hit_hang = (hit_speed*math.sin(hit_angle*(math.pi/180))+math.sqrt((hit_speed*math.sin(hit_angle*(math.pi/180)))**2 + (2 * 9.8 * 1))) / 9.8
hit_x = x
hit_y = y
hit_dist = math.sqrt(  (centroid_map_x[stadium+str(hit_location)]-x)**2  + (centroid_map_y[stadium+str(hit_location)]-y)**2)
catch_odds = logisticRegr.predict_proba(np.array((hit_dist,hit_hang)).reshape(1,-1))[0][1]

catchy = round(catch_odds*100,1)
if example['out'] == 1:
    if catchy < 20:
        emojis = '💎💎💎💎💎'
    elif catchy < 40:
        emojis = '💎💎💎💎'
    elif catchy < 60:
        emojis = '💎💎💎'
    elif catchy < 80:
        emojis = '💎💎'
    elif catchy <= 100:
        emojis = '🌽'
elif example['out'] == 0:
    if catchy > 80:
        emojis = '🍀🍀🍀🍀🍀'
    elif catchy > 60:
        emojis = '🍀🍀🍀🍀'
    elif catchy > 40:
        emojis = '🍀🍀🍀'
    elif catchy > 20:
        emojis = '🍀🍀'
    elif catchy >= 0:
        emojis = '💥'

dff2 = pd.read_csv('22_input.csv')
df_filter = dff2[dff2['stadium'] == stadium].reset_index(drop=True)
distances = []
locations = []
xs = []
ys = []
for location in df_filter['location'].unique().tolist():
    x = centroid_map_x[dff2['stadium'][0]+str(location)]
    y = centroid_map_y[dff2['stadium'][0]+str(location)]
    for i in range(len(df_filter)):
        xs.append(dff2['x'][i])
        ys.append(dff2['y'][i])
        locations.append(location)
        distances.append(math.sqrt(  (x-dff2['x'][i])**2  + (y-dff2['y'][i])**2))

df_show = pd.DataFrame(columns = ['distance'], data = distances)
df_show['hang_time'] = hit_hang
df_show['location'] = locations
df_show['x'] = xs
df_show['y'] = ys
df_show['catch_odds'] = logisticRegr.predict_proba(df_show[['distance', 'hang_time']])[::, 1]
df_show = df_show.sort_values(by='catch_odds')
if hit_hang <= 3.5:
    df_show = df_show[df_show['distance'] < 45]
else:
    df_show = df_show[df_show['distances'] < 70]

img = Image.open(requests.get('https://raw.githubusercontent.com/Melverk/bsbl-images/main/fields/'+home_team.upper()+'.png', stream=True).raw)
image_link = 'https://img.mlbstatic.com/mlb-photos/image/upload/d_people:generic:headshot:67:current.png/w_500,q_auto:best/v1/people/'+str(fielder)+'/headshot/67/current'
img2 = Image.open(requests.get(image_link, stream=True).raw).convert('RGBA')
image_link3 = 'https://img.mlbstatic.com/mlb-photos/image/upload/d_people:generic:headshot:67:current.png/w_500,q_auto:best/v1/people/'+str(hitter)+'/headshot/67/current'
img3 = Image.open(requests.get(image_link3, stream=True).raw).convert('RGBA')
fig, ax = plt.subplots( figsize =(12, 10), facecolor='dimgray')
im = ax.scatter(df_show[df_show['location'] == int(hit_location)]['x'], df_show[df_show['location'] == int(hit_location)]['y'], c = df_show[df_show['location'] == int(hit_location)]['catch_odds'], cmap= 'Blues', linewidths = 0, s=800, alpha=0.5)


plt.text(hit_x-5, hit_y+3.5, "x", fontsize = 30, fontweight = 'bold', color = 'black')
plt.text(200, 34, '----------  ', fontsize = 25, fontweight = 'bold', color = 'black', backgroundcolor='dimgray')
plt.text(200, 10, away_team+': '+str(away_score)+'    ', fontsize = 25, fontweight = 'bold', color = 'black', backgroundcolor='dimgray')
plt.text(200, 25, home_team+': '+str(home_score)+'    ', fontsize = 25, fontweight = 'bold', color = 'black', backgroundcolor='dimgray')
plt.text(200, 45, outs+' Outs      ', fontsize = 25, fontweight = 'bold', color = 'black', backgroundcolor='dimgray')
plt.text(200, 60, inning+'       ', fontsize = 25, fontweight = 'bold', color = 'black', backgroundcolor='dimgray')
im2 = ax.scatter(x2, y2, color='white', s = z2, zorder=4)

plt.text(133, 187, 'Hang Time: '+str(round(hit_hang,1))+'s', fontsize = 14, fontweight = 'bold', color = 'black', backgroundcolor = 'dimgray', zorder =5)
plt.text(133, 197, 'Distance: '+str(int(example['distance']*2.192))+' ft.', fontsize = 14, fontweight = 'bold', color = 'black', backgroundcolor = 'dimgray', zorder =5)

plt.text(65, 197, result_play, fontsize = 30, fontweight = 'bold', color = 'black', backgroundcolor = 'dimgray', zorder =5)

plt.text(200, 168, '----------  ', fontsize = 25, fontweight = 'bold', color = 'black', backgroundcolor='dimgray')
try:
    plt.text(200, 132, str(int(example['totalDistance']))+' ft.   ', fontsize = 20, fontweight = 'bold', color = 'black', backgroundcolor='dimgray')
except:
    plt.text(200, 132, '? ft.   ', fontsize = 20, fontweight = 'bold', color = 'black', backgroundcolor='dimgray')
plt.text(200, 145, str(round(hit_speed/.44704,1))+' mph   ', fontsize = 20, fontweight = 'bold', color = 'black', backgroundcolor='dimgray')
plt.text(200, 158, str(round(hit_angle,1))+'°        ', fontsize = 20, fontweight = 'bold', color = 'black', backgroundcolor='dimgray')
plt.text(200, 178, 'Catch Odds    ', fontsize = 17, fontweight = 'bold', color = 'black', backgroundcolor='dimgray')
if int(catch_odds*100) != 100:
    plt.text(200, 195, str(int(catch_odds*100))+'%      ', fontsize = 35, fontweight = 'bold', color = 'black', backgroundcolor='dimgray')
else:
    plt.text(200, 195, str(100)+'%      ', fontsize = 35, fontweight = 'bold', color = 'black', backgroundcolor='dimgray')

plt.imshow(img,zorder=0, extent=[-20,270,250,-50])
plt.imshow(img2,zorder=1, extent=[5,55,70,5], )
plt.imshow(img3,zorder=1, extent=[5,55,195,130])
ax.set_ylim(200, 0)
ax.set_xlim(0,250)
ax.set_xticks([])
ax.set_yticks([])
plt.title(description, fontsize=14)
cb =fig.colorbar(im, ax=ax, shrink=0.8, aspect=15,  drawedges='N', )
cb.ax.set_yticklabels(["{:.1%}".format(i) for i in cb.get_ticks()], fontsize=16, weight='bold')
print(description)
print('https://baseballsavant.mlb.com/sporty-videos?playId='+playId)
plt.axis('off')
plt.show()

C:\Users\btmyr\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


KeyError: 'distances'

In [177]:
df_show

,distances,hang_time,location,x,y,catch_odds
0,19.781801,2.670136,7,76.97,85.46,0.55
2865,20.346954,2.670136,8,106.54,68.77,0.55
2869,31.650123,2.670136,8,156.95,80.33,0.55
2870,41.554309,2.670136,8,165.76,56.29,0.55
2872,11.847550,2.670136,8,122.77,59.58,0.55
...,...,...,...,...,...,...
1434,4.139981,2.670136,7,68.49,103.56,0.55
1436,32.903669,2.670136,7,65.01,136.77,0.55
1438,15.994163,2.670136,8,111.05,67.90,0.55
1442,30.071264,2.670136,8,110.69,96.12,0.55
